In [1]:
import math

# Add the numbers of respirators in excess or in lack to a list of dictionaries as new key value pairs
def Add_Number_Of_Respirators(hospitalsList):
    
    for i in range(len(hospitalsList)):
        respirators = hospitalsList[i]["Dh"] - hospitalsList[i]["Rh"]
        if (respirators != 0):
            hospitalsList[i]["Dh-Rh"] = respirators
        else:
            del hospitalsList[i] ## delete hospitals that have enough respirators but cannot supply any (supply - demand = 0)
        
    return hospitalsList

# Seperate list of hospitals into 2 lists of dictionaries: one for hospitals in demand of respirators
# and another for hospitals that can supply respirators
def Find_Consumer_Supplier_Hospitals(hospitalsList):
    
    consumerHospitals = []
    supplierHospitals = []
    
    for i in range(len(hospitalsList)):
        if (hospitalsList[i]["Dh-Rh"] > 0):
            consumerHospitals.append(hospitalsList[i])
        else:
            supplierHospitals.append(hospitalsList[i])
    
    return consumerHospitals, supplierHospitals
    
def Find_Objective_Function(consumerHospitals, supplierHospitals):
    
    equation = ""
    
    # create all the different possible combinations of supplier/consumer hospitals (e.g. H31 means hospital 3 supplies hospital 1)
    # and generate equation
    for i in range(len(consumerHospitals)):
        for j in range(len(supplierHospitals)):
            equation += ("100000 * " +
                        str((math.sqrt((consumerHospitals[i]["Xh"] - supplierHospitals[j]["Xh"])**2 +
                                  (consumerHospitals[i]["Yh"] - supplierHospitals[j]["Yh"])**2))) +
                        "*H" + str(supplierHospitals[j]["ID"]) + str(consumerHospitals[i]["ID"]) +
                        " + 200000*Y" + str(supplierHospitals[j]["ID"]) + str(consumerHospitals[i]["ID"]) +
                        " + \n")
            
    equation = equation[:-4]
    print("Minimize: Objective function\n")
    print(equation)

def Find_Decision_Variables(consumerHospitals, supplierHospitals):
    
    supplierHospitals_list = []
    consumerHospitals_list = []
    
    for i in range(len(supplierHospitals)):
        supplierHospitals_list.append(supplierHospitals[i]["ID"])
    
    for i in range(len(consumerHospitals)):
        consumerHospitals_list.append(consumerHospitals[i]["ID"])
    
    print("Decision Variables:\n")
    print("Hij = Number of respirators transported from hospital i to hospital j with i = " +
          str(supplierHospitals_list) + " and j = " + str(consumerHospitals_list))
    print("Yij = 1 if transportation occurs from i to j, 0 if transportation does not occur from i to j with i = " +
          str(supplierHospitals_list) + " and j = " + str(consumerHospitals_list))

def Find_Constraints(consumerHospitals, supplierHospitals):
    
    decision_variable_list = []
    binary_variable_list = []
    supply = 0
    demand = 0
    total_number_hospitals = len(supplierHospitals) + len(consumerHospitals)
    
    constraints = "Constraints: \n\n"
        
    for i in range(len(supplierHospitals)):
        supply += supplierHospitals[i]["Dh-Rh"]
        demand = 0 ## 0 because we must calculate demand ONE time, it does not depend on how many times the 1st loop iterates
        for j in range(len(consumerHospitals)):
            decision_variable_list.append("H" + str(supplierHospitals[i]["ID"]) + str(consumerHospitals[j]["ID"]))
            binary_variable_list.append("Y" + str(supplierHospitals[i]["ID"]) + str(consumerHospitals[j]["ID"]))
            demand += consumerHospitals[j]["Dh-Rh"]
        
    if (supply * (-1) < demand):
        constraints += "\tSolving the problem might be infeasible since demand is greater than supply.\n\n"
        
    # constraints for supplier hospitals
    constraints += "\t1) Constraints for supplier hospitals:\n"
    for i in range(len(supplierHospitals)):
        constraints += ("\t\tHospital H" + str(supplierHospitals[i]["ID"]) + ": ")
        for j in range(len(consumerHospitals)):
            constraints += ("- H" + str(supplierHospitals[i]["ID"]) + str(consumerHospitals[j]["ID"]) + " ")
                
        if (supply * (-1) == demand):
            constraints += ("= " + str(supplierHospitals[i]["Dh-Rh"]) + "\n")
        elif (supply * (-1) > demand):
            constraints += (">= " + str(supplierHospitals[i]["Dh-Rh"]) + "\n")
        else:
            constraints += ("<= " + str(supplierHospitals[i]["Dh-Rh"]) + "\n")
        
            
    # constraints for consumer hospitals
    constraints += "\n\t2) Constraints for consumer hospitals:\n"
    for i in range(len(consumerHospitals)):
        constraints += ("\t\tHospital H" + str(consumerHospitals[i]["ID"]) + ": ")
        for j in range(len(supplierHospitals)):
            constraints += ("H" + str(supplierHospitals[j]["ID"]) + str(consumerHospitals[i]["ID"]) + " + ")
            
        constraints = constraints[:-3]
            
        if (supply * (-1) == demand):
            constraints += (" = " + str(consumerHospitals[i]["Dh-Rh"]) + "\n")
        elif (supply * (-1) > demand):
            constraints += (" >= " + str(consumerHospitals[i]["Dh-Rh"]) + "\n")
        else:
            constraints += (" <= " + str(consumerHospitals[i]["Dh-Rh"]) + "\n")
        
    # constraints for binary variables
    constraints += "\n\t3) Constraints for binary variables:\n"
    for i in range(len(decision_variable_list)):
        constraints += "\t\t" + str(decision_variable_list[i]) + " <= 1000000 * " + str(binary_variable_list[i] + "\n")
        
    # General constraints
    constraints += ("\n\t4) General constraints:\n")
    constraints += ("\n\t\t" + str(decision_variable_list) + " >= 0 and integers. \n\t")
    constraints += ("\n\t\t" + str(binary_variable_list) + " are binary variables.")
            
    print(constraints)
    
        
def Run_Program (hospitals):
    newList = Add_Number_Of_Respirators(hospitals)
    consumerHospitals, supplyHospitals = Find_Consumer_Supplier_Hospitals(newList)
    Find_Decision_Variables(consumerHospitals, supplyHospitals)
    print()
    Find_Objective_Function(consumerHospitals, supplyHospitals)
    print()
    Find_Constraints(consumerHospitals, supplyHospitals)